# Work With Daeploy From a Notebook

This example will show you how to create and deploy a service straight from a notebook.

## Logging in

To log in to your host, first log in using a terminal with username + password and generate a token with `daeploy token`. Use this token for future log-ins from the notebook. This way you do not have to show your username and password. You can set a time limit to the token for better security.

We start by importing some packages and defining the host and token as global variables

In [1]:
import requests
import json
import time

HOST = "http://localhost"  # Change to your host
TOKEN = ""  # Add your token here

.. note:: In jupyter notebooks you can call terminal commands by starting a row with `!`. We can use this with the Daeploy CLI to log in to a manager and use all the functionality that we are used to from the command line. You can prepend an argument with `$` to use a python variable in your command.

In [2]:
!daeploy login --host $HOST --token $TOKEN

Changed host to http://localhost


## Writing the Service

The service code should be collected in a single cell decorated with the `service_cell` [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html?highlight=magic). We start by importing it from `daeploy.notebook`. 

.. warning:: The function has to imported directly for it to work. IPython will not recognize `%%daeploy.notebook.service_cell`.

In [3]:
from daeploy.notebook import service_cell

The magic command has an argument for the project name of the service, which is the directory where all the files will be placed.

For example `%%service_cell my_service` will create a directory with a `service.py` file with the contents of the cell and a `.s2i/environment` file that is required by Daeploy. If the files already exist, only `serivce.py` will be changed.

In [4]:
%%service_cell notebook_service

import logging

import numpy as np

from daeploy import service
from daeploy.communication import notify, Severity

logger = logging.getLogger(__name__)


@service.entrypoint
def sqrt(number: float) -> float:
    return np.sqrt(number)


if __name__ == "__main__":
    service.run()

'Created project notebook_service. Saved service.py'

If the service needs any packages besides `daeploy` that are not included in the python standard library you can add them with the `service_requirements` magic:

In [5]:
from daeploy.notebook import service_requirements

In [6]:
%%service_requirements notebook_service
numpy

'Saved requirements.txt'

## Deploy the Service

We can deploy the service like we normally would with the CLI using `daeploy deploy`

In [7]:
!daeploy deploy notebook_service 1.0.0 notebook_service
time.sleep(5)  # Wait a moment for the service to start

Active host: http://localhost
Deploying service...
Service deployed successfully
MAIN    NAME              VERSION    STATUS    RUNNING
------  ----------------  ---------  --------  -----------------------------------
*       notebook_service  1.0.0      running   Running (since 2021-03-10 08:54:01)



## Test the Deployed Service

We can test the service straight from the notebook using the `requests` package.

In [8]:
response = requests.post(
    url=f"{HOST}/services/notebook_service/sqrt",
    json={"number": 2},
    headers={"Authorization": f"Bearer {TOKEN}"}
)
assert response.status_code == 200, f"{response.status_code}, {response.text}"
response.json()

1.4142135623730951

## Kill the Service

When we are finished we can kill the service.

In [9]:
!daeploy kill notebook_service 1.0.0 --yes

Active host: http://localhost
MAIN    NAME              VERSION    STATUS    RUNNING
------  ----------------  ---------  --------  -----------------------------------
*       notebook_service  1.0.0      running   Running (since 2021-03-10 08:54:01)

Service notebook_service 1.0.0 killed.
